In [11]:
import os
from glob import glob

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.client import device_lib

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [18]:
print(tf.config.list_physical_devices('GPU'))

[]


In [19]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6878276661807658806
xla_global_id: -1
]


2022-05-25 18:22:46.244771: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-05-25 18:22:46.247449: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


# tf input pipline
## tf.data.dataset 만들기

In [ ]:
train_path = "./dataset/main_directory/"
test_path = "./dataset/test/"

INPUT_SHAPE = (224,224,3)
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

tf.random.set_seed(7)
np.random.seed(7)

In [ ]:
def parse_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img,INPUT_SHAPE[:2])
    return img

def make_dataset(filepaths,labels):
    filenames_ds = tf.data.Dataset.from_tensor_slices(filepaths)
    images_ds = filenames_ds.map(
        parse_image,
        num_parallel_calls=AUTOTUNE
    )
    labels_ds = tf.data.Dataset.from_tensor_slices(labels)
    ds = tf.data.Dataset.zip((images_ds, labels_ds))
    return ds

def configure_for_performance(ds):
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.repeat()
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

In [ ]:
classes = os.listdir(train_path)
num_classes = len(classes)
print("number of classes :", num_classes)

filenames = glob(train_path+'*/*')
num_images = len(filenames)
print("number of images :", num_images)

print(filenames[0])
print(filenames[0].split(os.sep)[-2])
print()

np.random.shuffle(filenames)
labels = [classes.index(fn.split(os.sep)[-2]) for fn in filenames]

print("data 5 preview")
for path,label in zip(filenames[:5],labels[:5]):
    print(path)
    print("class :",classes[label])

# 학습셋, 검증셋으로 나누기

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(
    filenames, labels, test_size=0.2,
    stratify=labels, random_state=1
)
num_train = len(train_x)
num_val = len(val_x)
print("number of training data :", num_train)
print("number of validation data :", num_val)

print("data 5 preview")
for path,label in zip(train_x[:5],train_y[:5]):
    print(path)
    print("class :",classes[label])

# tf dataset 객체 만들기

In [ ]:
train_ds = make_dataset(train_x, train_y)

# tf.data.dataset 확인

for img, label in train_ds.take(5):
    # print(img.numpy().shape)
    img = img.numpy()
    # print(img.min(), img.max())
    
    img = img.astype(np.uint8)
    idx = (label.numpy())
    
    plt.imshow(img), plt.axis('off')
    plt.title(classes[idx])
    plt.show()

In [ ]:
val_ds = make_dataset(val_x, val_y)

In [ ]:
# 모델 성능 높이기
train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)

tf.data.experimental.save(train_ds, "train_ds", compression="GZIP")
tf.data.experimental.save(val_ds, "val_ds", compression="GZIP")

train_ds = tf.data.experimental.load("train_ds", compression="GZIP")
val_ds = tf.data.experimental.load("val_ds", compression="GZIP")

# data augmentation layer
## evaluate() 또는 predict() 호출 시에는 자동으로 비활성화 된다.

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
    keras.layers.experimental.preprocessing.RandomTranslation(0.1,0.1),
])

# model

In [ ]:
transfer_model = keras.applications.Xception(
    input_shape= INPUT_SHAPE,
    include_top= False,
    weights= 'imagenet',
)
transfer_model.trainable = True
# transfer_model.summary()

In [ ]:
finetune_model = keras.Sequential([
    keras.Input(shape=INPUT_SHAPE),
    data_augmentation,
    keras.layers.experimental.preprocessing.Rescaling(1./255),
    transfer_model,
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(2000, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(num_classes, activation='softmax')
])
# finetune_model.summary()

finetune_model.compile(loss='sparse_categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                       metrics='accuracy')                    

# batch 데이터셋 확인

for batch in train_ds.take(1):
    for i in range(5):
        img = batch[0][i]
        # print(img.numpy().shape)
        img = img.numpy()
        # print(img.min(), img.max())
        
        img = img.astype(np.uint8)
        
        label = batch[1][i]
        idx = (label.numpy())
        
        plt.imshow(img), plt.axis('off')
        plt.title(classes[idx])
        plt.show()

In [ ]:
train_step = num_train//BATCH_SIZE
val_step = num_val//BATCH_SIZE

filepath = "./model/{epoch:03d}-{val_loss:.4f}.h5"
check_point = keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_best_only = True,
    verbose = True
)
early_stop_point = keras.callbacks.EarlyStopping(patience=50)

hist = finetune_model.fit(
    train_ds, epochs=1000,
    steps_per_epoch=train_step,
    validation_data=val_ds,
    validation_steps=val_step,
    callbacks = [check_point, early_stop_point]
)

In [ ]:
plt.plot(hist.history["loss"], c='b', label="train_loss")
plt.plot(hist.history["val_loss"], c='r', label="val loss")
plt.legend()
plt.show()